In [1]:
from fastai.tabular import *

In [2]:
path = untar_data(URLs.ADULT_SAMPLE)
path

PosixPath('/home/paperspace/.fastai/data/adult_sample')

In [3]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


In [4]:
procs = [FillMissing, Categorify, Normalize]

In [5]:
valid_idx = range(len(df)-2000, len(df))

In [6]:
dep_var = '>=50k'
cat_names = [
    'workclass', 
    'education', 
    'marital-status', 
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country'
]

In [7]:
data = TabularDataBunch.from_df(path, 
                                df, 
                                dep_var, 
                                valid_idx=valid_idx,
                                procs=procs, 
                                cat_names=cat_names
                               )
print(data.train_ds.cont_names)

['education-num', 'hours-per-week', 'capital-gain', 'capital-loss', 'fnlwgt', 'age']


In [8]:
(cat_x, cont_x), y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[ 5 10  3 13  1  5  2 40  1]
 [ 5 12  3  4  1  5  2 40  1]
 [ 5  5  5  9  4  5  2  9  1]
 [ 5 12  3  9  1  5  2 40  1]
 [ 7 15  3 11  1  5  2 40  1]]
[[ 1.145728  0.368169 -0.145922 -0.216787 -1.065956 -0.265377]
 [-0.421569 -0.035789  0.241564 -0.216787 -1.256181 -0.851194]
 [-2.772515 -0.035789 -0.145922 -0.216787  2.019832 -1.144103]
 [-0.421569  1.984002 -0.145922 -0.216787  0.23183  -0.558286]
 [ 1.929376  0.772127 -0.145922 -0.216787  0.083637  0.833029]]
[1 0 0 0 1]


In [13]:
learn = tabular_learner(data, 
                        layers=[200,100], 
                        emb_szs={'native-country': 10},
                        metrics=accuracy
                       )
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.322014,0.311394,0.852000


In [14]:
learn.predict(df.iloc[0])

(Category 1, tensor(1), tensor([0.1409, 0.8591]))